In [1]:
import time
import requests
from bs4 import BeautifulSoup
import json
import re
import time
import faker
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from dateutil import parser

In [2]:
from collections import Counter

In [3]:
fake = faker.Faker(locale='en')

In [4]:
ibm_headers = {
    'User-Agent': fake.chrome(),
    'accept-language': 'en-US,en;q=0.9',
    'pragma': 'np-cache',
    'origin': 'https://www.ibm.com',
    'referer': 'https://www.ibm.com/'
}

In [5]:
API_URL = "http://jobsapi-internal.m-cloud.io/api/stjobbulk"

In [6]:
search_payload = {
    'organization': 2242,
    'limitkey': '4A8B5EF8-AA98-4A8B-907D-C21723FE4C6B',
    'facet': 'publish_to_cws:true',
    'fields': 'title,id,update_date,primary_country,description,primary_category,level,url,brand'
}

In [7]:
result = requests.get(API_URL, headers=ibm_headers, params=search_payload)
result

<Response [200]>

In [8]:
result.json()['totalHits'], len(result.json()['queryResult'])

(9137, 9137)

In [9]:
result.json()['queryResult'][0].keys()

dict_keys(['id', 'title', 'primary_country', 'description', 'primary_category', 'level', 'brand', 'url', 'update_date'])

In [10]:
Counter([j['primary_category'] for j in result.json()['queryResult']]).most_common(10)

[('Technical Specialist', 3601),
 ('Consultant', 2045),
 ('Finance', 1075),
 ('Architect', 499),
 ('Sales', 490),
 ('Software Development & Support', 329),
 ('Project Management', 300),
 ('Human Resources', 172),
 ('Other', 145),
 ('Enterprise Operations', 111)]

In [11]:
clear_string = lambda x: re.sub(' +', ' ', re.sub('<.*?>', ' ', x)).strip()

In [17]:
about_pattern = re.compile(r"Introduction(.+?)Your Role and Responsibilities", flags=re.DOTALL|re.IGNORECASE)
responsibilities_pattern = re.compile(r"Your Role and Responsibilities(.+?)Required Technical and Professional Expertise", flags=re.DOTALL|re.IGNORECASE)
qualifications_pattern = re.compile(r"Required Technical and Professional Expertise(.+?)Preferred Technical and Professional Expertise", flags=re.DOTALL|re.IGNORECASE)

In [30]:
job_dicts = []
for job_info in tqdm(result.json()['queryResult']):
    full_description = clear_string(job_info['description'])

    about = re.search(about_pattern, full_description)
    if about:
        about = about.group(1)

    responsibilities = re.search(responsibilities_pattern, full_description)
    if responsibilities:
        responsibilities = responsibilities.group(1)

    qualifications = re.search(qualifications_pattern, full_description)
    if qualifications:
        qualifications = qualifications.group(1)

    if not qualifications and not responsibilities and not about:
        print(f"Skipping vacancy {job_info['url']}")

    job_dict = {
        'title': job_info['title'],
        'internal_id' : job_info['id'],
        'url': job_info['url'],
        'description': about,
        'responsibilities': responsibilities,
        'qualifications': qualifications,
        'company': 'IBM',
        'grade': job_info['level'],
        'category': job_info['primary_category'],
        'publish_date': parser.parse(job_info['update_date'])
    }
    job_dicts.append(job_dict)
len(job_dicts)

  5%|▍         | 432/9137 [00:00<00:04, 2118.84it/s]

Skipping vacancy https://careers.ibm.com/job/15232232/-ibm-pm-it-it-tokyo-jp/
Skipping vacancy https://careers.ibm.com/job/15232235/-ibm-pm-it-it-tokyo-jp/
Skipping vacancy https://careers.ibm.com/job/15232237/-ibm-pm-it-it-tokyo-jp/
Skipping vacancy https://careers.ibm.com/job/15270425/-ibm-japan-tax-manager-tokyo-jp/
Skipping vacancy https://careers.ibm.com/job/15280931/-ibm-japan-ibm-cloud-solution-architect-tokyo-jp/
Skipping vacancy https://careers.ibm.com/job/15072653/fullstack-java-per-lima-pe/
Skipping vacancy https://careers.ibm.com/job/15017893/-pm-tokyo-jp/
Skipping vacancy https://careers.ibm.com/job/15037807/l-der-de-integraci-n-bogota-co/
Skipping vacancy https://careers.ibm.com/job/14742685/-ibm-seller-business-development-executive-ibm-research-tokyo-jp/
Skipping vacancy https://careers.ibm.com/job/14326157/-hybrid-cloud-t-research-tokyo-student-intern-intern-trl-ibm-japan-kawasaki-jp/
Skipping vacancy https://careers.ibm.com/job/14969281/-data-ai-expert-lab-tokyo-jp/
S

  9%|▉         | 850/9137 [00:00<00:04, 1974.38it/s]

Skipping vacancy https://careers.ibm.com/job/15826502/desarrollador-net-buenos-aires-ar/
Skipping vacancy https://careers.ibm.com/job/15900419/desarrollador-java-fullstack-lima-pe/
Skipping vacancy https://careers.ibm.com/job/15900420/desarrollador-java-fullstack-lima-pe/
Skipping vacancy https://careers.ibm.com/job/15900421/desarrollador-java-fullstack-lima-pe/
Skipping vacancy https://careers.ibm.com/job/15901438/desarrollador-java-fullstack-lima-pe/
Skipping vacancy https://careers.ibm.com/job/15901439/desarrollador-android-lima-pe/
Skipping vacancy https://careers.ibm.com/job/13339387/-ise-storage-engineer-chiba-jp/
Skipping vacancy https://careers.ibm.com/job/15915718/-ise-chiba-jp/
Skipping vacancy https://careers.ibm.com/job/15915722/-ise-chiba-jp/
Skipping vacancy https://careers.ibm.com/job/15915723/-ise-chiba-jp/
Skipping vacancy https://careers.ibm.com/job/16086167/consultora-or-sap-sd-remote/
Skipping vacancy https://careers.ibm.com/job/16116390/desarrollador-full-stack-net

 14%|█▍        | 1260/9137 [00:00<00:03, 1996.48it/s]

...

Skipping vacancy https://careers.ibm.com/job/15636356/desenvolvedor-a-ou-consultor-a-salesforce-remoto-remote/


 97%|█████████▋| 8898/9137 [00:04<00:00, 1959.19it/s]

Skipping vacancy https://careers.ibm.com/job/15405032/desenvolvedor-de-aplicativos-back-end-s-o-paulo-br/
Skipping vacancy https://careers.ibm.com/job/16216505/application-developer-java-web-technologies-s-o-paulo-br/
Skipping vacancy https://careers.ibm.com/job/16322279/est-gio-em-vendas-s-o-paulo-br/
Skipping vacancy https://careers.ibm.com/job/16435949/assistente-financeiro-junior-hortol-ndia-br/
Skipping vacancy https://careers.ibm.com/job/16435950/assistente-financeiro-junior-hortol-ndia-br/
Skipping vacancy https://careers.ibm.com/job/15094404/java-portal-developer-remote/
Skipping vacancy https://careers.ibm.com/job/15094407/tech-lead-remote/
Skipping vacancy https://careers.ibm.com/job/15100725/ingeniero-a-devops-remote/
Skipping vacancy https://careers.ibm.com/job/15100726/desarrollador-a-frontend-remote/
Skipping vacancy https://careers.ibm.com/job/15605717/especialista-senior-en-pruebas-de-control-de-calidad-remote/
Skipping vacancy https://careers.ibm.com/job/15828697/data-

100%|██████████| 9137/9137 [00:04<00:00, 2019.30it/s]


9137

In [31]:
snapshot = pd.DataFrame(job_dicts)
snapshot.sample(5)

,title,internal_id,url,description,responsibilities,qualifications,company,grade,category,publish_date
6154,Service Associate - Onboarding,16371781,https://careers.ibm.com/job/16371781/service-a...,"At IBM, work is more than a job - it's a call...","As Service Associate -Onboarding, you are res...",Graduate / Post Graduate in any stream Excell...,IBM,Entry Level,Human Resources,2022-08-05 14:48:31+00:00
2799,Application Consultant,16052767,https://careers.ibm.com/job/16052767/applicati...,IBM Application Consultants work directly wit...,1. Identify and analyze user requirements 2. ...,1. Above 8 years’ work experience as a Applic...,IBM,Professional,Consultant,2022-07-14 05:34:37+00:00
1522,Full Stack Developer,15886156,https://careers.ibm.com/job/15886156/full-stac...,"As a Full Stack Python/AWS Developer, you wil...","Expertus, an IBM Company, is looking for Full...","Programming language : Python, Java, HTLM, CS...",IBM,Professional,Technical Specialist,2022-07-19 05:29:21+00:00
6949,Associate Partner - Identity and Access Manage...,16289768,https://careers.ibm.com/job/16289768/associate...,Informationen und Daten zählen zu den wichtig...,Als Associate Partner für Identity and Access...,Sie haben mindestens 8 Jahre Projekterfahrung...,IBM,Professional,Consultant,2022-08-11 02:40:40+00:00
4678,Open Source Toolchain Developer,16421430,https://careers.ibm.com/job/16421430/open-sour...,Software Developers at IBM are the backbone o...,IBM Power Systems Enterprise Servers are base...,6+ years of overall IT experience 1+ years of...,IBM,Professional,Software Development & Support,2022-08-05 18:46:21+00:00


In [32]:
snapshot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9137 entries, 0 to 9136
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype                  
---  ------            --------------  -----                  
 0   title             9137 non-null   object                 
 1   internal_id       9137 non-null   int64                  
 2   url               9137 non-null   object                 
 3   description       8598 non-null   object                 
 4   responsibilities  8598 non-null   object                 
 5   qualifications    8598 non-null   object                 
 6   company           9137 non-null   object                 
 7   grade             9137 non-null   object                 
 8   category          9137 non-null   object                 
 9   publish_date      9137 non-null   datetime64[ns, tzutc()]
dtypes: datetime64[ns, tzutc()](1), int64(1), object(8)
memory usage: 714.0+ KB


In [33]:
snapshot['publish_date'] = snapshot['publish_date'].apply(lambda x: x.replace(tzinfo=None))
snapshot['publish_date'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 9137 entries, 0 to 9136
Series name: publish_date
Non-Null Count  Dtype         
--------------  -----         
9137 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 71.5 KB


In [34]:
current_date = datetime.now().strftime('%d-%m-%Y')
current_date

'11-08-2022'

In [35]:
snapshot.to_csv(f'../data/ibm/{current_date}.csv')
snapshot.to_excel(f'../data/ibm/{current_date}.xlsx', engine='xlsxwriter')

#### Проверка на единичной вакансии

In [26]:
full_description = clear_string(result.json()['queryResult'][200]['description'])
full_description

"Introduction At IBM, we know that transformation drives change, and it is in our DNA to continuously transform. Our Business Transformation Consultants help drive large scale change that spans IBM business units and geographies. As a part of this team, you will have the unique opportunity to be a part of driving IBM’s ongoing transformation as a company that continuously innovates and leads the market. Your Role and Responsibilities Your day in the role will include... Managing and developing applications using BluePrism. You will come with... Experience on BluePrism. How we’ll help you grow... You’ll have access to all the technical and management training courses to grow your expertise. You’ll learn directly from expert developers in the field; our team leads love to mentor. You have the opportunity to work in many different areas to determine what really excites you. Required Technical and Professional Expertise RPA Product Specialist with Technical Deployment Experience in one or 

In [27]:
re.search(about_pattern, full_description).group(1)

' At IBM, we know that transformation drives change, and it is in our DNA to continuously transform. Our Business Transformation Consultants help drive large scale change that spans IBM business units and geographies. As a part of this team, you will have the unique opportunity to be a part of driving IBM’s ongoing transformation as a company that continuously innovates and leads the market. '

In [28]:
re.search(responsibilities_pattern, full_description).group(1)

' Your day in the role will include... Managing and developing applications using BluePrism. You will come with... Experience on BluePrism. How we’ll help you grow... You’ll have access to all the technical and management training courses to grow your expertise. You’ll learn directly from expert developers in the field; our team leads love to mentor. You have the opportunity to work in many different areas to determine what really excites you. '

In [29]:
re.search(qualifications_pattern, full_description).group(1)

' RPA Product Specialist with Technical Deployment Experience in one or more leading RPA technologies (e.g. Blue Prism, Automation Anywhere, UIPath etc). Prior client consulting experience in RPA technologies would be an added advantage '